In [9]:
from pydub import AudioSegment
from vosk import Model, KaldiRecognizer
import wave
import json
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import os
print(os.getcwd())  # This will print the current directory

audio_file_loc = "C:\\Users\\ssromerogon\\Documents\\vscode_working_dir\\meeting_minutes_processing\\"
os.chdir(audio_file_loc)
print(os.getcwd())  # This will print the current directory



C:\Users\ssromerogon\Documents\vscode_working_dir\meeting_minutes_processing
C:\Users\ssromerogon\Documents\vscode_working_dir\meeting_minutes_processing


In [ ]:
# 1. Convert MP3 to WAV using pydub
audio_file = "scdc 11_13_24.mp3"
wav_file = "scdc_11_13_24.wav"

# Convert MP3 to WAV
audio = AudioSegment.from_mp3(audio_file)
audio.export(wav_file, format="wav")


In [ ]:

# 2. Speech-to-Text Conversion using Vosk
model = Model("path/to/vosk-model")  # Download from https://alphacephei.com/vosk/models
wf = wave.open(wav_file, "rb")

if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() not in [8000, 16000]:
    raise ValueError("Audio file must be WAV format mono PCM.")

recognizer = KaldiRecognizer(model, wf.getframerate())
recognizer.SetWords(True)
text = ""

while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if recognizer.AcceptWaveform(data):
        result = json.loads(recognizer.Result())
        text += result.get("text", "") + " "

wf.close()

# 3. Natural Language Processing
nltk.download('punkt')
nltk.download('stopwords')

# Tokenization
sentences = sent_tokenize(text)
words = word_tokenize(text)

# Stop Word Removal
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in words if word.lower() not in stop_words]

# Stemming
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in filtered_words]

# 4. Meeting Minute Generation
def generate_minutes(sentences):
    minutes = []
    for sentence in sentences:
        if "action item" in sentence.lower() or "decision" in sentence.lower():
            minutes.append(sentence)
    return minutes

meeting_minutes = generate_minutes(sentences)

# 5. Save Minutes to a Text File
output_file = "meeting_minutes_vosk.txt"
with open(output_file, "w") as f:
    for line in meeting_minutes:
        f.write(line + "\n")

print("Meeting minutes saved to", output_file)
